A notebook demonstrating sampling elevations using a number of rasters. This is in essence replacing some common ArcGIS tools

Neil Symington
neil.symington@ga.gov.au


In [1]:
from sqlite3 import dbapi2 as sqlite
import pandas as pd
import fiona
from shapely.geometry import Polygon, shape
from shapely import wkt
import numpy as np
from hydrogeol_utils import spatial_functions, SNMR_utils
from geophys_utils._netcdf_point_utils import NetCDFPointUtils
import matplotlib.pyplot as plt
import os
import math
import sqlalchemy as db
from sqlalchemy import create_engine, event

import rasterio


In [15]:
# First open the SNMR database

# Our next tak is to bring the SNMR data in 

# Connect to the SNMR database

# Now we want to know the screened interval and other spatial information so we open up the 
# borehole database


SPATIALITE_PATH = r'C:\mod_spatialite-4.3.0a-win-amd64'

# Add spatialite dll to path
os.environ['PATH'] = SPATIALITE_PATH + ';' + os.environ['PATH']


DB_PATH = r"\\prod.lan\active\proj\futurex\East_Kimberley\Data\Processed\Geophysics\NMR\SNMR\East_Kimberley_SNMR.sqlite"
        
engine = db.create_engine('sqlite:///' + DB_PATH, module=sqlite)

@event.listens_for(engine, 'connect')
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')


connection = engine.connect()

query = """

SELECT
              s.*
              
FROM 

             sites as s
             ;"""

df_sites = pd.read_sql_query(query, connection, index_col = 'site_id')

# Add an elevation column

df_sites['elevation_mAHD'] = np.nan

In [16]:
# Define our elevation rasters

lid_10m = r"\\prod.lan\active\proj\futurex\East_Kimberley\Working\SharedWorkspace\LiDAR\LiDAR_final\EK_LiDAR_cubic_10x10.tif"

srtm = r"\\prod.lan\active\proj\futurex\East_Kimberley\Data\Processed\ElevationDepth\Elevation\DEM_1sec\demh1sv1ek_z52.tif"

In [17]:
lid_src = rasterio.open(lid_10m)

srtm_src = rasterio.open(srtm)

lid_bounds = lid_src.bounds

srtm_bounds = srtm_src.bounds

In [18]:
# Iterate through the sites and add the raster value to the dataframe
# if it is within the extent of the raster
for index, row in df_sites.iterrows():
    # get the coords
    x, y = row.mid_X, row.mid_Y
    # Check lidar
    if spatial_functions.point_within_bounds(x,y, lid_bounds):
        
        elev = next(lid_src.sample(np.array([[x,y]])))[0]
        
        df_sites.at[index, 'elevation_mAHD'] = elev
    # Check srtm
    elif spatial_functions.point_within_bounds(x,y, srtm_bounds):
        
        elev = next(srtm_src.sample(np.array([[x,y]])))[0]
        
        df_sites.at[index, 'elevation_mAHD'] = elev
    # Otherwise print for further investigation
    else:
        print(row)
    # TO protect against nulls we will make all < -10 mAHD values nan
    if elev < -10:
        df_sites.at[index, 'elevation_mAHD'] = np.nan
        